In [3]:
import requests

url = "https://celestrak.org/NORAD/elements/stations.txt"
response = requests.get(url)

with open("../data/raw/iss_tle.txt", "w") as f:
    f.write(response.text)

print("ISS TLE data downloaded and saved")


ISS TLE data downloaded and saved
